# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [46]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
# nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
import pickle

In [25]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("Messages", con=engine)
# df = df.head(10)
X = df['message']  # Get Message column
Y = df.iloc[:, 4:] # Extract Category data

### 2. Write a tokenization function to process your text data

In [28]:
def tokenize(text):
    
    """
    Function: 
    split text into words and return the root form of the words
    
    INPUT:
    text - message
      
    OUTPUT:
    clean_tokens - a list of the root form of words
    """
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip() 
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [29]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.2, random_state = 45)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [56]:
def category_report(model, X_test, y_test):
    '''
    Function:
    This function to use the classification model to predict the category, and 
    generate classification report
    
    Input: 
    Model - Classification model
    X_test - messages 
    y_test - categories 
    
    Output: 
    Print the classification report
    '''
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print('Category {}: {}'.format(i+1,col))
        print(classification_report(y_test[col], y_pred[:, i]))



In [61]:
# test
category_names = y_test.columns
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [34]:
category_report(pipeline, X_test, y_test)

Category 1: related
              precision    recall  f1-score   support

           0       0.72      0.27      0.39      1198
           1       0.81      0.97      0.88      4002
           2       0.69      0.20      0.32        44

    accuracy                           0.80      5244
   macro avg       0.74      0.48      0.53      5244
weighted avg       0.79      0.80      0.76      5244

Category 2: request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      4335
           1       0.89      0.44      0.59       909

    accuracy                           0.89      5244
   macro avg       0.89      0.71      0.76      5244
weighted avg       0.89      0.89      0.88      5244

Category 3: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5214
           1       0.00      0.00      0.00        30

    accuracy                           0.99      5244
   macro avg    

C:\Users\wangx\.conda\envs\python36\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Category 10: child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5244

    accuracy                           1.00      5244
   macro avg       1.00      1.00      1.00      5244
weighted avg       1.00      1.00      1.00      5244

Category 11: water
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4918
           1       0.90      0.22      0.35       326

    accuracy                           0.95      5244
   macro avg       0.93      0.61      0.66      5244
weighted avg       0.95      0.95      0.94      5244

Category 12: food
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      4678
           1       0.85      0.43      0.57       566

    accuracy                           0.93      5244
   macro avg       0.89      0.71      0.76      5244
weighted avg       0.93      0.93      0.92      5244

Category 1

### 6. Improve your model
Use grid search to find better parameters. 

In [35]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001B2FDD8D8C8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [40]:
parameters =  {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 200],
        'clf__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, param_grid = parameters)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [41]:
cv.fit(X_train, y_train)
category_report(cv, X_test, y_test)

Category 1: related
              precision    recall  f1-score   support

           0       0.73      0.26      0.39      1198
           1       0.81      0.97      0.88      4002
           2       0.71      0.23      0.34        44

    accuracy                           0.80      5244
   macro avg       0.75      0.49      0.54      5244
weighted avg       0.79      0.80      0.76      5244

Category 2: request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      4335
           1       0.89      0.44      0.59       909

    accuracy                           0.89      5244
   macro avg       0.89      0.72      0.76      5244
weighted avg       0.89      0.89      0.88      5244

Category 3: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5214
           1       0.00      0.00      0.00        30

    accuracy                           0.99      5244
   macro avg    

C:\Users\wangx\.conda\envs\python36\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.92      1.00      0.96      4827
           1       0.73      0.05      0.10       417

    accuracy                           0.92      5244
   macro avg       0.83      0.53      0.53      5244
weighted avg       0.91      0.92      0.89      5244

Category 6: medical_products
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      4990
           1       0.94      0.06      0.11       254

    accuracy                           0.95      5244
   macro avg       0.95      0.53      0.54      5244
weighted avg       0.95      0.95      0.93      5244

Category 7: search_and_rescue
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      5086
           1       0.67      0.03      0.05       158

    accuracy                           0.97      5244
   macro avg       0.82      0.51      0.52      5244
weighted avg   

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4785
           1       0.91      0.38      0.54       459

    accuracy                           0.94      5244
   macro avg       0.93      0.69      0.75      5244
weighted avg       0.94      0.94      0.93      5244

Category 31: storm
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4774
           1       0.79      0.47      0.59       470

    accuracy                           0.94      5244
   macro avg       0.87      0.73      0.78      5244
weighted avg       0.94      0.94      0.93      5244

Category 32: fire
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5195
           1       0.00      0.00      0.00        49

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.98      0.99    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [44]:
model = AdaBoostClassifier()

pipeline_optimize = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(model))
    ])

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.2, random_state = 45)
pipeline_optimize.fit(X_train, y_train)

parm1 = 'tfidf__use_idf'
parm2 = 'clf__estimator__n_estimators'
parm3 = 'clf__estimator__learning_rate'

parameters =  {
        parm1: (True, False),
        parm2: [50, 100],
        parm3: [1, 2]
}

cv = GridSearchCV(pipeline_optimize, param_grid = parameters)
cv.fit(X_train, y_train)
category_report(cv, X_test, y_test)

Category 1: related
              precision    recall  f1-score   support

           0       0.57      0.23      0.32      1198
           1       0.80      0.95      0.87      4002
           2       0.36      0.11      0.17        44

    accuracy                           0.78      5244
   macro avg       0.57      0.43      0.45      5244
weighted avg       0.74      0.78      0.74      5244

Category 2: request
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      4335
           1       0.76      0.54      0.63       909

    accuracy                           0.89      5244
   macro avg       0.84      0.75      0.79      5244
weighted avg       0.88      0.89      0.88      5244

Category 3: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5214
           1       0.00      0.00      0.00        30

    accuracy                           0.99      5244
   macro avg    


Category 28: other_infrastructure
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      5020
           1       0.32      0.10      0.15       224

    accuracy                           0.95      5244
   macro avg       0.64      0.55      0.57      5244
weighted avg       0.93      0.95      0.94      5244

Category 29: weather_related
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      3794
           1       0.85      0.66      0.74      1450

    accuracy                           0.87      5244
   macro avg       0.87      0.81      0.83      5244
weighted avg       0.87      0.87      0.87      5244

Category 30: floods
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      4785
           1       0.87      0.57      0.69       459

    accuracy                           0.95      5244
   macro avg       0.91      0.78      0.83 

### 9. Export your model as a pickle file

In [62]:
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.